In [104]:
import random

import networkx as nx
import numpy as np

from utils_cv import *
from utils import compute_degree
from utils import *

In [105]:

def encode_to_cv(n, edges):
    """mã hóa đồ thị n đỉnh thành chuỗi bit"""
    g = nx.complete_graph(n)
    g_edges = list(g.edges)

    bit = [0] * len(g_edges)
    for i in range(len(g_edges)):
        if g_edges[i] in edges:
            bit[i] = 1
    return bit


def decode_to_graph(n, cv):
    g = nx.complete_graph(n)
    g_edges = list(g.edges)
    edges = []
    for i in range(len(g_edges)):
        if cv[i] == 1:
            edges.append(g_edges[i])
    return edges

In [106]:
def make_graph_connected(G):
    """Biến đồ thị không liên thông trở thành liên thông"""
    # Tìm các thành phần liên thông của đồ thị
    subgraphs = list(nx.connected_components(G))
    # print(subgraphs)

    # Nếu đồ thị đã liên thông, không cần thực hiện gì thêm
    if len(subgraphs) == 1:
        return G

    # Thêm các cạnh để kết nối các thành phần liên thông
    for i in range(len(subgraphs) - 1):
        nodes1 = subgraphs[i]
        nodes2 = subgraphs[i + 1]
        node1 = nodes1.pop()
        node2 = nodes2.pop()
        G.add_edge(node1, node2)

    return G

In [107]:

def fix_cycle(nx_graph):
    """Đầu vào là một đồ thị liên thông"""
    while is_cycle(nx_graph):
        c = nx.find_cycle(nx_graph)
        nx_graph.remove_edge(c[1][0], c[1][1])
    return nx_graph


def is_cycle(nx_graph):
    if not nx.is_tree(nx_graph) and nx.is_connected(nx_graph):
        return True
    else:
        return False

In [108]:

def calculate_fitness(cv_sequence, degree_constrained, distances_table):
    n = compute_n(len(cv_sequence))
    edges = decode_to_graph(n, cv_sequence)
    T = nx.from_edgelist(edges)
    degrees = list(compute_degree(T).values())
    # print(degrees)

    # Check if the degrees more than target degrees
    if any(x > degree_constrained for x in degrees):
        return 9999999

    cost = 0
    for edge in edges:
        cost = cost + get_distance(edge, distances_table)
    return cost


# distance_table = get_distance_table("")
# calculate_fitness([1, 0, 0, 0, 1, 0, 0, 1, 0, 0], 2, distance_table)

In [109]:
def crossover(parent1, parent2, crossover_rate=1):
    offspring1 = parent1[:]
    offspring2 = parent2[:]
    if random.random() < crossover_rate:
        crossover_point = random.randint(1, len(parent1) - 2)
        offspring1[crossover_point:], offspring2[crossover_point:] = offspring2[crossover_point:], offspring1[
                                                                                                   crossover_point:]
    return offspring1, offspring2

In [110]:

def mutate(individual, mutation_rate=0.1):
    if random.random() < mutation_rate:
        idx1, idx2 = random.sample(range(len(individual)), 2)
        individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
    return individual


def mutate_pop(population, mutation_rate):
    new_pop = []
    for individual in population:
        if random.random() < mutation_rate:
            idx1, idx2 = random.sample(range(len(individual)), 2)
            individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
            new_pop.append(individual)
    return new_pop


In [111]:
def get_new_pop(population, population_size, degree_constrained, distances_table):
    fitness_values = [calculate_fitness(individual, degree_constrained, distances_table) for
                      individual in population]
    sorted_fitness = sorted(fitness_values)
    index_dict = {val: idx for idx, val in enumerate(fitness_values)}
    # Lấy danh sách index của population đã được sắp xếp theo fitness
    pop_index = [index_dict[val] for val in sorted_fitness]
    # Lấy index của [population_size] phần tử đầu (có cost nhỏ nhất)
    new_pop_index = pop_index[:population_size]
    new_pop = [population[i] for i in new_pop_index]
    return new_pop

In [112]:
def repair_degree(degree_constrained, edges):
    # Tạo đồ thị cây từ danh sách cạnh
    G = nx.Graph(edges)
    subgraphs = []
    # Kiểm tra giới hạn bậc của mỗi đỉnh
    for v in G.nodes():
        if G.degree(v) > degree_constrained:
            # Nếu có đỉnh có giới hạn bậc quá n, thực hiện sửa cây
            # Tìm các đỉnh kề của đỉnh này
            neighbors = list(G.neighbors(v))
            # print("v, neighbor", v, neighbors)
            # Xóa các cạnh kết nối đỉnh này với các đỉnh kề
            for i in range(len(neighbors) - degree_constrained):
                G.remove_edge(neighbors[i], v)

            subgraphs = list(nx.connected_components(G))
    subgraphs = convert_lset_to_llist(subgraphs)
    # print(subgraphs)
    degree = compute_degree(G)
    # print(degree)
    subgraphs = sort_sublists(subgraphs, degree)
    # print(subgraphs)
    while len(subgraphs) > 1:
        # Ghép 2 đồ thị đầu tiên
        G.add_edge(subgraphs[0][0], subgraphs[1][0])
        # Tính toán lại subgraph và bậc đồ thị sau khi ghép
        subgraphs = list(nx.connected_components(G))
        subgraphs = convert_lset_to_llist(subgraphs)
        degree = compute_degree(G)
        subgraphs = sort_sublists(subgraphs, degree)
        # print(subgraphs)

    # Trả về danh sách cạnh của cây đã được sửa
    return G


def repair_pop(pop, degree_constrained):
    for i in range(len(pop)):
        # print("individual", i)
        n = compute_n(len(pop[i]))
        # print(n)
        edges = decode_to_graph(n, pop[i])
        # print(edges)
        # Làm đồ thị liên thông
        G = make_graph_connected(nx.from_edgelist(edges))
        # print(G.edges())
        # Xóa chu trình
        G = fix_cycle(G)
        print(is_cycle(G))
        G = repair_degree(degree_constrained, G.edges())
        pop[i] = encode_to_cv(n,G.edges())
    return pop

In [113]:
def run_ga(n, degree_constrained, distances_table, population_size=50, crossover_rate=0.8, mutation_rate=0.1,
           max_generations=50):
    population = [gen_bit(n) for _ in range(population_size)]
    # print(population)
    for generation in range(max_generations):
        print(population)
        # Tính một mảng fitness value của population
        fitness_values = [calculate_fitness(cv_sequence, degree_constrained, distances_table) for
                          cv_sequence in population]
        print(fitness_values)
        new_population = []
        while len(new_population) < population_size:
            i, j = np.random.choice(range(population_size), size=2, replace=False,
                                    p=np.array(fitness_values) / sum(fitness_values))
            parent1 = population[i]
            parent2 = population[j]
            offspring1, offspring2 = crossover(parent1, parent2)
            new_population.append(offspring1)
            new_population.append(offspring2)
        for i in range(len(new_population)):
            mutate(new_population[i])
        repair_pop(new_population, degree_constrained)
        population = population + new_population
        population = get_new_pop(population, population_size, degree_constrained, distances_table)

    return population

In [114]:
degree_constrained = 2
n = 5
distances_table = get_distance_table("data/7_wi29.csv")

population = run_ga(n, degree_constrained, distances_table)
print(population[0])
print(calculate_fitness(population[0], degree_constrained, distances_table))

[[0, 0, 1, 0, 0, 1, 0, 1, 1, 0], [1, 1, 0, 1, 0, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, 1, 1, 0, 1, 0], [0, 0, 0, 0, 1, 1, 0, 0, 1, 1], [0, 0, 1, 0, 0, 0, 1, 1, 1, 0], [1, 0, 0, 1, 1, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 1, 0, 1, 1, 0], [0, 1, 0, 1, 0, 0, 1, 0, 0, 1], [1, 0, 0, 0, 1, 1, 0, 0, 1, 0], [0, 0, 1, 1, 1, 0, 1, 0, 0, 0], [0, 1, 0, 0, 1, 0, 1, 0, 0, 1], [1, 0, 1, 1, 1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 1, 0, 1, 0, 0], [0, 0, 1, 1, 0, 1, 0, 0, 0, 1], [1, 0, 1, 0, 0, 1, 0, 0, 1, 0], [1, 0, 0, 0, 0, 1, 1, 1, 0, 0], [0, 0, 0, 1, 0, 1, 0, 0, 1, 1], [0, 0, 0, 0, 0, 1, 1, 0, 1, 1], [0, 1, 0, 1, 1, 1, 0, 0, 0, 0], [1, 0, 0, 1, 1, 0, 0, 1, 0, 0], [0, 0, 0, 1, 1, 1, 1, 0, 0, 0], [0, 0, 0, 1, 1, 0, 1, 0, 0, 1], [0, 1, 1, 0, 0, 0, 0, 1, 0, 1], [1, 0, 0, 0, 0, 1, 1, 0, 1, 0], [0, 1, 1, 0, 0, 1, 1, 0, 0, 0], [1, 0, 0, 1, 0, 1, 0, 0, 0, 1], [1, 0, 0, 0, 0, 1, 0, 0, 1, 1], [0, 1, 0, 1, 0, 0, 0, 1, 0, 1], [0, 0, 1, 1, 1, 0, 0, 0, 1, 0], [0, 1, 1, 0, 0, 0, 0, 1, 0, 1], [1, 0, 0, 0, 1, 1, 0, 0, 0, 1], [1, 0, 

NetworkXPointlessConcept: G has no nodes.